## Game Content
* Game detail data "https://gol.gg/game/stats/24969/page-game/"

* reference 
    1. crawler table method https://zhuanlan.zhihu.com/p/351603761

* Try coding

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import urllib
import urllib.request
from urllib.request import urlopen
import csv
import pandas as pd
import numpy as np 
def team_lose_win(team_soup):
    team_Text = team_soup[0].getText()
    team_sp = team_Text.split(' - ')
    team_name = (team_sp[0].split('\n'))[1]
    team_result = team_sp[1]
    return [team_name,team_result]
def soupToList(soup,trReduce,tdReduce):
    table=soup.find_all("tr")
    datai=[]
    for i in range(len(table)+trReduce):
        dataj=[]
        for j in range(len(table[1].find_all("td"))+tdReduce):
            dataj.append((table[i-trReduce].find_all("td"))[j-tdReduce].getText())
        datai.append(dataj)
    return datai
def get_computation_data(computation_ID):

    #################################################
    headers="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    urpage="https://gol.gg/game/stats/{}/page-fullstats/".format(computation_ID)
    url=urllib.request.Request(urpage,headers={"user-Agent":headers})
    page = urlopen(url)
    soup=bs(page,"html.parser")
    gameList=soupToList(soup,-1,0)

    gameDf=pd.DataFrame(gameList)
    gameDfT = gameDf.T

    gameAry=np.array(gameList)
    gameAryT=gameAry.T 
    gameAryT[0]
    gameDfT.columns = gameAryT[0]
    filterFrist=(gameDfT['Player']!='Player')
    gameDfT1=gameDfT[filterFrist]

    table=soup.find_all("tr")
    hero=(table[0]).find_all('th')
    heroList=[]
    for i in range(len(hero)-1):
        heroList.append(hero[i+1].find('img')['alt'])
    heroDf=pd.DataFrame(data=heroList,columns=['hero'])

    gameDfT1.reset_index(drop=True,inplace=True)
    gameDfT2=pd.concat([heroDf,gameDfT1],axis=1)

    ######################################################
    headers="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    urpage="https://gol.gg/game/stats/{}/page-game/".format(computation_ID)
    url=urllib.request.Request(urpage,headers={"user-Agent":headers})
    page = urlopen(url)
    page_game_soup=bs(page,"html.parser")

    blue_team_soup = page_game_soup.find_all("div",class_='col-12 blue-line-header')
    red_team_soup = page_game_soup.find_all("div",class_='col-12 red-line-header')
    blue_team_data = team_lose_win(blue_team_soup)
    red_team_data = team_lose_win(red_team_soup)

    ########################################################
    team_list=[blue_team_data[0],blue_team_data[0],blue_team_data[0],blue_team_data[0],blue_team_data[0],red_team_data[0],red_team_data[0],red_team_data[0],red_team_data[0],red_team_data[0]]
    win_lose_list=[blue_team_data[1],blue_team_data[1],blue_team_data[1],blue_team_data[1],blue_team_data[1],red_team_data[1],red_team_data[1],red_team_data[1],red_team_data[1],red_team_data[1]]
    gameDfT2['TEAM'] = team_list
    gameDfT2['WIN_LOSE'] = win_lose_list
    return gameDfT2
    

get_computation_data("24969")
   

,hero,Player,Role,Level,Kills,Deaths,Assists,KDA,CS,CS in Team's Jungle,...,Time ccing others,Total Time CC Dealt,Total damage taken,Total Time Spent Dead,Consumables purchased,Items Purchased,Shutdown bounty collected,Shutdown bounty lost,TEAM,WIN_LOSE
0,Kennen,Azhi,TOP,,3,3,5,2.7,344,8,...,18,,23852,,,,,,Berjaya Dragons,LOSS
1,Graves,Enso,JUNGLE,,2,4,11,3.3,297,138,...,20,,27497,,,,,,Berjaya Dragons,LOSS
2,LeBlanc,1116,MID,,3,3,4,2.3,322,20,...,15,,27049,,,,,,Berjaya Dragons,LOSS
3,Ashe,Keres,ADC,,5,3,9,4.7,449,43,...,39,,20493,,,,,,Berjaya Dragons,LOSS
4,Thresh,Do1u1u,SUPPORT,,2,3,9,3.7,33,0,...,30,,23504,,,,,,Berjaya Dragons,LOSS
5,Gangplank,Rest,TOP,,6,1,7,13,390,15,...,16,,38362,,,,,,J Team,WIN
6,Trundle,Hana,JUNGLE,,0,3,11,3.7,208,105,...,33,,49508,,,,,,J Team,WIN
7,Syndra,Nestea,MID,,7,2,2,4.5,372,33,...,17,,19470,,,,,,J Team,WIN
8,Jhin,LilV,ADC,,3,3,7,3.3,440,37,...,18,,17052,,,,,,J Team,WIN
9,Nautilus,Woody,SUPPORT,,0,6,11,1.8,41,0,...,34,,19289,,,,,,J Team,WIN


## All game list

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import re
import csv

def get_Tournaments_link(season_button_ID):
    driver_path = 'C:\chromedriver-win64\chromedriver.exe'
    driver = webdriver.Chrome(executable_path=driver_path)
    url = "https://gol.gg/tournament/list/"
    driver.get(url)
    # 這裡可能需要等待頁面載入完成
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "result_tab")))
    button = driver.find_element(By.ID, season_button_ID)
    button.click()

    # 再次等待頁面載入完成
    time.sleep(2)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "result_tab")))

    # 取得新頁面的 HTML 內容
    new_page_html = driver.page_source
    soup = BeautifulSoup(new_page_html, 'html.parser')
    driver.quit()
    
    result_tab=soup.find('div', id='result_tab')
    result_tab_td=result_tab.find_all('td')
    #result_tab_td[1].find('a')['href']
    Tournaments_list = []
    for i in range(len(result_tab_td)):
        try:
            x=result_tab_td[i].find('a')['href']
            #print(x)
            Tournaments_url='https://gol.gg/tournament/tournament-matchlist/{}'.format(x[19:])
            Tournaments_list.append(Tournaments_url)
        except:
            pass
    return Tournaments_list

def get_Tournaments_allLink(season_button_ID_list):
    Tournaments_allList=[]
    for i in range(len(season_button_ID_list)):
        Tournaments_allList = Tournaments_allList + get_Tournaments_link(season_button_ID_list[i])
    return Tournaments_allList
def get_gamelist1Tournament(TournamentURL):
    driver_path = 'C:\chromedriver-win64\chromedriver.exe'
    driver = webdriver.Chrome(executable_path=driver_path) # 請提供您的 ChromeDriver 路徑
    driver.get(TournamentURL)

    text_to_wait_for = "Worlds Main Event 2023  "
    element_locator = (By.XPATH, f"//*[contains(text(), '{text_to_wait_for}')]")
    try:
        element = WebDriverWait(driver, 10).until(
            EC.text_to_be_present_in_element(element_locator, text_to_wait_for)
        )
        print(f"Element with text '{text_to_wait_for}' is present on the page.")
        # 在这里可以进一步操作该元素
    except :
        print(f"Timed out waiting for element with text '{text_to_wait_for}' to appear on the page")

    new_page_html1 = driver.page_source

    # 使用 Beautiful Soup 解析新頁面的內容
    gamelist1page_soup = BeautifulSoup(new_page_html1, 'html.parser')
    gamelist1page_td=gamelist1page_soup.find_all('td')
    gamelist1page_soup = BeautifulSoup(new_page_html1, 'html.parser')
    gamelist1page_td=gamelist1page_soup.find_all('td')

    gamelist1page_lst=[]
    for i in range(len(gamelist1page_td)):
        try:
            y = gamelist1page_td[i].find('a')['href']
            numbers = re.findall(r'\d+', y)
            gamelist1page_lst.append(numbers[0])
        except:
            pass
    return gamelist1page_lst
def get_gamelistAllTournament(Tournaments_allLink):
    gamelistAllTournament=[]
    for i in range(len(Tournaments_allLink)):
        gamelistAllTournament=gamelistAllTournament+get_gamelist1Tournament(Tournaments_allLink[i])
    return gamelistAllTournament

Tournaments_allLink_list=get_Tournaments_allLink(['link-S7','link-S6','link-S5','link-S4','link-S3'])

C:\Users\kid50\AppData\Local\Temp\ipykernel_19152\3505612867.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


In [2]:
my_list=get_gamelistAllTournament(Tournaments_allLink_list)
# 指定输出的文件名
csv_file = "output1.csv"

# 使用 csv 模块写入 CSV 文件
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # 写入每一行数据
    writer.writerow(my_list)

print(f"CSV 文件 '{csv_file}' 已创建成功。")

C:\Users\kid50\AppData\Local\Temp\ipykernel_19152\3505612867.py:50: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path) # 請提供您的 ChromeDriver 路徑


Timed out waiting for element with text 'Worlds Main Event 2023  ' to appear on the page
Timed out waiting for element with text 'Worlds Main Event 2023  ' to appear on the page
Timed out waiting for element with text 'Worlds Main Event 2023  ' to appear on the page
Timed out waiting for element with text 'Worlds Main Event 2023  ' to appear on the page
Timed out waiting for element with text 'Worlds Main Event 2023  ' to appear on the page
Timed out waiting for element with text 'Worlds Main Event 2023  ' to appear on the page
Timed out waiting for element with text 'Worlds Main Event 2023  ' to appear on the page
Timed out waiting for element with text 'Worlds Main Event 2023  ' to appear on the page
Timed out waiting for element with text 'Worlds Main Event 2023  ' to appear on the page
Timed out waiting for element with text 'Worlds Main Event 2023  ' to appear on the page
Timed out waiting for element with text 'Worlds Main Event 2023  ' to appear on the page
Timed out waiting for

In [6]:
import csv
csv_file = "output.csv"

# 使用 csv 模块写入 CSV 文件
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # 写入每一行数据
    writer.writerow(my_list)

print(f"CSV 文件 '{csv_file}' 已创建成功。")

CSV 文件 'output.csv' 已创建成功。


In [7]:
csv_file = "output.csv"

# 用于存储读取的数据的列表
data_list = []

# 使用 csv 模块读取 CSV 文件
with open(csv_file, mode='r') as file:
    reader = csv.reader(file)
     # 遍历每一行数据
    for row in reader:
        data_list.append(row)

In [4]:
from get_lol_game_info import get_game_info

# Player List
* URL : https://gol.gg/players/list/season-S14/split-Pre-Season/tournament-ALL/

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyperclip
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyperclip
import time

# 設定瀏覽器驅動程序的路徑
# 請根據您使用的瀏覽器（Chrome、Firefox等）和操作系統進行更改
driver_path = 'C:\chromedriver-win64\chromedriver.exe'
driver = webdriver.Chrome(executable_path=driver_path)

# 打開網頁
url = 'https://gol.gg/players/list/season-S14/split-Pre-Season/tournament-ALL/'
driver.get(url)

# 找到並點擊按鈕
button = driver.find_element_by_css_selector('[data-clipboard-target="#copy_div"]')
button.click()

# 等待一些時間，確保內容已複製到剪貼板
time.sleep(2)

# 使用 pyperclip 獲取剪貼板中的內容
copied_text = pyperclip.paste()

# 現在 copied_text 包含了複製的內容
print(copied_text)

# 關閉瀏覽器
driver.quit()


C:\Users\kid50\AppData\Local\Temp\ipykernel_4800\1897591255.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)
C:\Users\kid50\AppData\Local\Temp\ipykernel_4800\1897591255.py:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  button = driver.find_element_by_css_selector('[data-clipboard-target="#copy_div"]')


Player	Country	Games	Win rate	KDA	Avg kills	Avg deaths	Avg assists	CSM	GPM	KP%	DMG%	DPM	VSPM	Avg WPM	Avg WCPM	Avg VWPM	GD@15	CSD@15	XPD@15	FB %	FB Victim	Penta Kills	Solo Kills
1xn	 CN	3	66.7%	6.3	4.3	2	8.3	9.4	464	60.9%	24.9	641	-	-	-	-	-	-	-	-	-	-	-
369	 CN	8	62.5%	3.3	4.1	2.9	5.5	8.2	411	49.6%	23.2	679	-	-	-	-	-	-	-	-	-	-	-
AKi	 CN	11	54.5%	3.5	2	2.5	7	5.7	344	72.8%	10.9	242	-	-	-	-	-	-	-	-	-	-	-
Able	 CN	3	33.3%	5.5	2.3	1.3	5	10	449	43.9%	32.2	753	-	-	-	-	-	-	-	-	-	-	-
Ahn	 CN	4	75%	7.2	9.5	2.3	6.8	9.7	521	83.7%	32.2	907	-	-	-	-	-	-	-	-	-	-	-
Angel	 CN	3	33.3%	6.8	2.3	1.3	6.7	9.1	408	72.6%	20.2	485	-	-	-	-	-	-	-	-	-	-	-
Assum	 CN	11	54.5%	4.6	3.8	2.2	6.3	9	436	65.7%	24.4	519	-	-	-	-	-	-	-	-	-	-	-
Beichuan	 CN	3	66.7%	2.2	3.3	4.7	7	6.5	392	54.5%	15.9	427	-	-	-	-	-	-	-	-	-	-	-
Bin	 CN	10	90%	4.5	3.8	2.1	5.7	8.8	439	51.2%	25.6	681	-	-	-	-	-	-	-	-	-	-	-
Breathe	 CN	7	57.1%	2.5	4.3	2.6	2.1	7.9	404	54.9%	27	579	-	-	-	-	-	-	-	-	-	-	-
Burdol	 KR	4	75%	5.3	3.5	2	7	7.5	400	49.9%	20.5	602	-	-

In [4]:
type(copied_text)

str

In [7]:
data_list = copied_text.split('\n')  # 根據實際情況選擇分隔符
print(data_list)

['Player\tCountry\tGames\tWin rate\tKDA\tAvg kills\tAvg deaths\tAvg assists\tCSM\tGPM\tKP%\tDMG%\tDPM\tVSPM\tAvg WPM\tAvg WCPM\tAvg VWPM\tGD@15\tCSD@15\tXPD@15\tFB %\tFB Victim\tPenta Kills\tSolo Kills\r', '1xn\t CN\t3\t66.7%\t6.3\t4.3\t2\t8.3\t9.4\t464\t60.9%\t24.9\t641\t-\t-\t-\t-\t-\t-\t-\t-\t-\t-\t-\r', '369\t CN\t8\t62.5%\t3.3\t4.1\t2.9\t5.5\t8.2\t411\t49.6%\t23.2\t679\t-\t-\t-\t-\t-\t-\t-\t-\t-\t-\t-\r', 'AKi\t CN\t11\t54.5%\t3.5\t2\t2.5\t7\t5.7\t344\t72.8%\t10.9\t242\t-\t-\t-\t-\t-\t-\t-\t-\t-\t-\t-\r', 'Able\t CN\t3\t33.3%\t5.5\t2.3\t1.3\t5\t10\t449\t43.9%\t32.2\t753\t-\t-\t-\t-\t-\t-\t-\t-\t-\t-\t-\r', 'Ahn\t CN\t4\t75%\t7.2\t9.5\t2.3\t6.8\t9.7\t521\t83.7%\t32.2\t907\t-\t-\t-\t-\t-\t-\t-\t-\t-\t-\t-\r', 'Angel\t CN\t3\t33.3%\t6.8\t2.3\t1.3\t6.7\t9.1\t408\t72.6%\t20.2\t485\t-\t-\t-\t-\t-\t-\t-\t-\t-\t-\t-\r', 'Assum\t CN\t11\t54.5%\t4.6\t3.8\t2.2\t6.3\t9\t436\t65.7%\t24.4\t519\t-\t-\t-\t-\t-\t-\t-\t-\t-\t-\t-\r', 'Beichuan\t CN\t3\t66.7%\t2.2\t3.3\t4.7\t7\t6.5\t392\t54.5%\t15.9

In [9]:
data_list2=[]
for i in range(len(data_list)):
    data_list_split=data_list[i].split('\t')
    data_list2.append(data_list_split)
data_list2

[['Player',
  'Country',
  'Games',
  'Win rate',
  'KDA',
  'Avg kills',
  'Avg deaths',
  'Avg assists',
  'CSM',
  'GPM',
  'KP%',
  'DMG%',
  'DPM',
  'VSPM',
  'Avg WPM',
  'Avg WCPM',
  'Avg VWPM',
  'GD@15',
  'CSD@15',
  'XPD@15',
  'FB %',
  'FB Victim',
  'Penta Kills',
  'Solo Kills\r'],
 ['1xn',
  ' CN',
  '3',
  '66.7%',
  '6.3',
  '4.3',
  '2',
  '8.3',
  '9.4',
  '464',
  '60.9%',
  '24.9',
  '641',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-\r'],
 ['369',
  ' CN',
  '8',
  '62.5%',
  '3.3',
  '4.1',
  '2.9',
  '5.5',
  '8.2',
  '411',
  '49.6%',
  '23.2',
  '679',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-\r'],
 ['AKi',
  ' CN',
  '11',
  '54.5%',
  '3.5',
  '2',
  '2.5',
  '7',
  '5.7',
  '344',
  '72.8%',
  '10.9',
  '242',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-\r'],
 ['Able',
  ' CN',
  '3',
  '33.3%',
  '5.5',
  '2.3',
  '1.3',
  '5',
  '10',
  '449',
  '43.9%',
  '32.